<a href="https://colab.research.google.com/github/wujciak/thesis_colab/blob/main/data_merge_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch1 data
W przypadku batch1 chińskiego datasetu należało dopasować oznakowanie do zdjęć oraz zanonimizować dane tak, aby nie zawierały imion i nazwisk pacjentów.

In [ ]:
import pandas as pd

csv_path = '/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch1_image/batch1_image_label.csv'
data = pd.read_csv(csv_path)

data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   patient_index  611 non-null    int64 
 1   patient_name   611 non-null    object
 2   histo_label    611 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 14.4+ KB


(   patient_index patient_name  histo_label
 0              1          曾令群            0
 1              2          沈甫祥            1
 2              3          沈惠芬            0
 3              4          沈利平            0
 4              5          沈美金            1,
 None)

In [ ]:
# Stworzenie mapy nazwa pacjenta -> unikalny ID
unique_ids = {name: f"ID{i+1}" for i, name in enumerate(data['patient_name'].unique())}

# Zastąpienie nazw pacjentów unikalnymi ID
data['patient_name'] = data['patient_name'].map(unique_ids)

# Podgląd zaktualizowanych danych
data.head(), unique_ids


(   patient_index patient_name  histo_label
 0              1          ID1            0
 1              2          ID2            1
 2              3          ID3            0
 3              4          ID4            0
 4              5          ID5            1,
 {'曾令群': 'ID1',
  '沈甫祥': 'ID2',
  '沈惠芬': 'ID3',
  '沈利平': 'ID4',
  '沈美金': 'ID5',
  '沈美玉': 'ID6',
  '沈兴海': 'ID7',
  '陈爱英': 'ID8',
  '陈福根': 'ID9',
  '陈钢': 'ID10',
  '陈海全': 'ID11',
  '陈嘉怡': 'ID12',
  '陈建平': 'ID13',
  '陈建英': 'ID14',
  '陈明强': 'ID15',
  '陈雨婷': 'ID16',
  '戴宇嫣': 'ID17',
  '单思营': 'ID18',
  '董茶英': 'ID19',
  '董雨强': 'ID20',
  '冯华英': 'ID21',
  '冯水英': 'ID22',
  '高林华': 'ID23',
  '顾金珍': 'ID24',
  '胡士颖': 'ID25',
  '胡先华': 'ID26',
  '黄秀艳': 'ID27',
  '贾必英': 'ID28',
  '姜秀芳': 'ID29',
  '蒋利群': 'ID30',
  '金卫珍': 'ID31',
  '金永明': 'ID32',
  '柯美芳': 'ID33',
  '刘珈伊': 'ID34',
  '芦金祥': 'ID35',
  '陆留生': 'ID36',
  '陆妹妹': 'ID37',
  '马建林': 'ID38',
  '马艳': 'ID39',
  '倪永青': 'ID40',
  '浦金华': 'ID41',
  '邱美娥': 'ID42',
  '邱永新': 'ID43',
  '阮彩芳': 'ID44'

In [ ]:
import os

# Ścieżka do folderu dataset
dataset_path = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch1_image/dataset"

# Pobranie listy plików w folderze
try:
    files = os.listdir(dataset_path)
except FileNotFoundError:
    files = []

# Podgląd plików i przygotowanie zmienionych nazw
renamed_files = []
if files:
    for file in files:
        # Wydobycie imienia pacjenta z nazwy pliku
        patient_name = file.split('_')[0]  # imię kończy się przed "_"

        # Mapowanie na unikalne ID
        new_id = unique_ids.get(patient_name, None)

        if new_id:
            # Utworzenie nowej nazwy pliku
            new_file_name = file.replace(patient_name, new_id)
            renamed_files.append((file, new_file_name))
        else:
            renamed_files.append((file, None))

renamed_files[:10]


[('丁小娟_002.Jpg', 'ID106_002.Jpg'),
 ('丁小娟_006.Jpg', 'ID106_006.Jpg'),
 ('丁小娟_001.Jpg', 'ID106_001.Jpg'),
 ('丁小娟_007.Jpg', 'ID106_007.Jpg'),
 ('丁小娟_005.Jpg', 'ID106_005.Jpg'),
 ('丁小娟_004.Jpg', 'ID106_004.Jpg'),
 ('丁秀娟_005.Jpg', 'ID542_005.Jpg'),
 ('丁秀娟_003.Jpg', 'ID542_003.Jpg'),
 ('丁思成_006.Jpg', 'ID215_006.Jpg'),
 ('丁思成_007.Jpg', 'ID215_007.Jpg')]

In [ ]:
import shutil

# Ścieżki docelowe dla benign i malignant
benign_path = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/data/benign_path"
malignant_path = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/data/malignant_path"

# Tworzenie folderów docelowych, jeśli nie istnieją
os.makedirs(benign_path, exist_ok=True)
os.makedirs(malignant_path, exist_ok=True)

# Przenoszenie plików do odpowiednich folderów
segregation_log = []

if files:
    for file, new_file_name in renamed_files:
        if new_file_name:
            patient_id = new_file_name.split('_')[0]

            # Pobierz etykietę histologiczną
            label = data.loc[data['patient_name'] == patient_id, 'histo_label'].values

            if len(label) == 1:
                label = label[0]
                src = os.path.join(dataset_path, file)

                # Określ ścieżkę docelową
                dst = os.path.join(benign_path if label == 0 else malignant_path, new_file_name)

                # Przenieś plik
                try:
                    shutil.move(src, dst)
                    segregation_log.append((file, dst))
                except Exception as e:
                    segregation_log.append((file, str(e)))

len(segregation_log), segregation_log[:10]  # Liczba przetworzonych plików i podgląd logu


(5984,
 [('丁小娟_002.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID106_002.Jpg'),
  ('丁小娟_006.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID106_006.Jpg'),
  ('丁小娟_001.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID106_001.Jpg'),
  ('丁小娟_007.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID106_007.Jpg'),
  ('丁小娟_005.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID106_005.Jpg'),
  ('丁小娟_004.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID106_004.Jpg'),
  ('丁秀娟_005.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID542_005.Jpg'),
  ('丁秀娟_003.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID542_003.Jpg'),
  ('丁思成_006.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/ID215_006.Jpg'),
  ('丁思成_007.Jpg',
   '/content/drive/MyDrive/Colab Notebooks/ch_dataset/benign_path/

# Batch2 data
To samo należało dokonać dla batch2 datasetu, tutaj dodatkowo należało usunąć jpg z raportami które nie były danymi treningowymi. Ostatecznie nie załączono batch2 do ostatecznego datasetu ze względu na zbyt duże zróżnicowanie względem pozostałych zdjęć.


In [ ]:
import os
import shutil
import pandas as pd

# Ścieżki - do zmiany według potrzeb
source_path = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch2_image/thyroid_3_10_month"
csv_path = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch2_image/batch2_image_label.csv"
target_path = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch2_image"

# Tworzenie folderów docelowych
target_benign = os.path.join(target_path, "benign")
target_malignant = os.path.join(target_path, "malignant")
os.makedirs(target_benign, exist_ok=True)
os.makedirs(target_malignant, exist_ok=True)

# Wczytanie pliku CSV
csv_file = pd.read_csv(csv_path)

# Iteracja po indeksach pacjentów
for _, row in csv_file.iterrows():
    patient_index = str(row['patient_index'])
    histo_label = row['histo_label']  # 0 = benign, 1 = malignant

    # Ścieżka do folderu pacjenta
    patient_folder = os.path.join(source_path, patient_index)

    if not os.path.exists(patient_folder):
        print(f"Folder {patient_folder} nie istnieje, pomijam.")
        continue

    # Wybór folderu docelowego
    target_folder = target_benign if histo_label == 0 else target_malignant

    # Iteracja po plikach w folderze pacjenta
    for file_name in os.listdir(patient_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            source_file = os.path.join(patient_folder, file_name)
            target_file = os.path.join(target_folder, f"{patient_index}_{file_name}")

            # Przeniesienie pliku
            shutil.copy2(source_file, target_file)

print("Przenoszenie zakończone.")



Folder /content/drive/MyDrive/Colab Notebooks/ch_dataset/batch2_image/thyroid_3_10_month/622 nie istnieje, pomijam.
Przenoszenie zakończone.


In [ ]:
# Anonimizacja danych
import os

# Ścieżki folderów
target_benign = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch2_image/benign"
target_malignant = "/content/drive/MyDrive/Colab Notebooks/ch_dataset/batch2_image/malignant"

# Funkcja do zmiany nazw plików
def anonymize_files(folder_path, prefix):
    file_counter = 1  # Start od 1
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Sprawdź, czy to plik obrazu
        if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            new_name = f"{prefix}{file_counter}.png"
            new_path = os.path.join(folder_path, new_name)
            os.rename(file_path, new_path)
            file_counter += 1

# Anonimizacja plików dla obu folderów
anonymize_files(target_benign, "b")
anonymize_files(target_malignant, "m")

print("Pliki zostały zanonimizowane.")


Pliki zostały zanonimizowane.


# Scalenie danych
Aktualnie batch1, batch2 chińskiego datasetu oraz algierski dataset są rozdzielone od siebie, ale każde z nich jest podzielone na foldery benign oraz malignant. Foldery batch1 i algerian scalono już ręcznie w dysku google.